In [3]:
# import pandas as pd
# import numpy as np
# from sklearn.svm import SVC

# essays = pd.read_csv('../data/processed/essays_with_topic_scores.csv', index_col=0)

In [4]:
# essays.head(2)

,essay_id,domain1_score,bad_if_kids_spend_too_much_time,doesnt_have_the_negative_exercise_effect,games_and_information,looking_at_websites_for_info,spend_time_looking_on_websites
0,1,8.0,0.000,0.562,0.149,0.269,0.018
1,2,9.0,0.108,0.000,0.000,0.885,0.000


In [5]:
# essays['length'] = essays.essay.str.len()

NameError: name 'essays1' is not defined

In [2]:
# import os
# import pandas as pd
# import numpy as np

# os.chdir('../utils')

# from helpers import lda_description_for_building, get_sample_essay, create_DF_from_tuple

# os.chdir('../notebooks')

# Topic names from LDA analysis
# TODO: Refactor out into it's own space
topic_names = {0: 'looking_at_websites_for_info',
               1: 'doesnt_have_the_negative_exercise_effect',
               2: 'spend_time_looking_on_websites',
               3: 'games_and_information',
               4: 'bad_if_kids_spend_too_much_time'}

# col_names = pd.DataFrame(columns=list(topic_names.values()))

# this is a bit time consuming - make the if statement True
# # if you want to create the dataframe yourself.
# if 0 == 1:

#     essays_with_topic_scores = process_topic_and_score_df(col_names)

# essays_with_topic_scores = essays_with_topic_scores.drop(['essay_set', 'essay', 'rater1_domain1', 'rater2_domain1', 'prompt', 'has_source_material', 'grade_7', 'grade_8', 'grade_10'], axis=1)
# essays_with_topic_scores.to_csv('../data/processed/essays_with_topic_scores.csv')

In [3]:
col_names = pd.DataFrame(columns=list(topic_names.values()))

In [4]:
col_names

,looking_at_websites_for_info,doesnt_have_the_negative_exercise_effect,spend_time_looking_on_websites,games_and_information,bad_if_kids_spend_too_much_time


In [19]:
%timeit

essays_with_topic_scores = process_topic_and_score_df(col_names)

NameError: name 'pd' is not defined

In [43]:
def process_topic_and_score_df(topic_names):
    
    processed_df = pd.DataFrame(columns=['essay_id', 'essay_set', 'essay', 'rater1_domain1', 'rater2_domain1',
       'domain1_score', 'prompt', 'has_source_material', 'grade_7', 'grade_8',
       'grade_10', 'bad_if_kids_spend_too_much_time',
       'doesnt_have_the_negative_exercise_effect', 'games_and_information',
       'looking_at_websites_for_info', 'spend_time_looking_on_websites'])
    for n in range(len(essays)):
        tuple = lda_description_for_building(get_sample_essay(n))
        topic_scores = create_DF_from_tuple(tuple)
        processed_topic_scores = pd.concat([topic_scores, topic_names], sort=True).drop_duplicates().reset_index(drop=True).fillna(0)
        indexNamesArr = processed_topic_scores.index.values
        indexNamesArr[0] = n
        merged_dfs = pd.concat([essays.loc[[n]], processed_topic_scores], axis=1, sort=False)
        processed_df = processed_df.append(merged_dfs)

    return processed_df

def get_sample_essay(essay_number):
    """
    retrieve a particular review index
    from the reviews file and return it
    """

    return list(it.islice(line_review(essay_set1_txt_filepath), essay_number, essay_number+1))[0]

def create_DF_from_tuple(tuple):
        
    return pd.DataFrame({k:v for k,*v in tuple})

def lda_description_for_building(essay_text, min_topic_freq=0.05):
        
    # parse the essay text with spaCy
    parsed_essay = nlp(essay_text)

    # lemmatize the text and remove punctuation and whitespace
    unigram_essay = [token.lemma_ for token in parsed_essay
                     if not punct_space_stop(token)]

    # apply the first-order and secord-order phrase models
    bigram_essay = bigram_model[unigram_essay]
    trigram_essay = trigram_model[bigram_essay]

    # create a bag-of-words representation
    essay_bow = trigram_dictionary.doc2bow(trigram_essay)

    # create an LDA representation
    essay_lda = lda[essay_bow]

    # sort with the most highly related topics first
    essay_lda = sorted(essay_lda)

    topics = []
    freqs = []
    for topic_number, freq in essay_lda:

        topics.append(topic_names[topic_number])
        freqs.append(round(freq, 3))
        
    # return topic and fre
    return list(zip(topics, freqs))

def punct_space_stop(token):
    """
    helper function to eliminate tokens
    that are pure punctuation, whitespace or stopwords
    """
    
    return token.is_punct or token.is_space or token.is_stop

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename,):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename), batch_size=100, n_threads=4):
        
        for sent in parsed_review.sents:
            yield ' '.join([token.lemma_ for token in sent if not punct_space_stop(token)])
            
def adding_stanford_nlp_groups_NER_to_stop_words():
    """
    helper funciton to add Stanford NLP Group NERs to spaCy stop words
    range of 0 - 15
    """

    for number in list(range(0, 16)):
        nlp.vocab['@ORGANIZATION' + str(number)].is_stop = True
        nlp.vocab['@PERSON' + str(number)].is_stop = True
        nlp.vocab['@CAPS' + str(number)].is_stop = True
        nlp.vocab['@LOCATION' + str(number)].is_stop = True
        nlp.vocab['@DATE' + str(number)].is_stop = True
        nlp.vocab['@TIME' + str(number)].is_stop = True
        nlp.vocab['@MONEY' + str(number)].is_stop = True
        nlp.vocab['@PERCENT' + str(number)].is_stop = True
        nlp.vocab['@MONTH' + str(number)].is_stop = True
        nlp.vocab['@EMAIL' + str(number)].is_stop = True
        nlp.vocab['@NUM' + str(number)].is_stop = True
        nlp.vocab['@DR' + str(number)].is_stop = True
        nlp.vocab['@CITY' + str(number)].is_stop = True
        nlp.vocab['@STATE' + str(number)].is_stop = True

def removing_stanford_nlp_groups_NER_from_stop_words():
    """
    helper funciton to remove Stanford NLP Group NERs from spaCy stop words
    range of 0 - 15
    """

    for number in list(range(0, 16)):
        nlp.vocab['@ORGANIZATION' + str(number)].is_stop = False
        nlp.vocab['@PERSON' + str(number)].is_stop = False
        nlp.vocab['@CAPS' + str(number)].is_stop = False
        nlp.vocab['@LOCATION' + str(number)].is_stop = False
        nlp.vocab['@DATE' + str(number)].is_stop = False
        nlp.vocab['@TIME' + str(number)].is_stop = False
        nlp.vocab['@MONEY' + str(number)].is_stop = False
        nlp.vocab['@PERCENT' + str(number)].is_stop = False
        nlp.vocab['@MONTH' + str(number)].is_stop = False
        nlp.vocab['@EMAIL' + str(number)].is_stop = False
        nlp.vocab['@NUM' + str(number)].is_stop = False
        nlp.vocab['@DR' + str(number)].is_stop = False
        nlp.vocab['@CITY' + str(number)].is_stop = False
        nlp.vocab['@STATE' + str(number)].is_stop = False

In [18]:
import os
import pandas as pd
import numpy as np
import codecs
import spacy
import itertools as it
import pandas as pd
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import Word2Vec

nlp = spacy.load('en_core_web_md')

In [19]:
adding_stanford_nlp_groups_NER_to_stop_words()

In [35]:
essay_set1_txt_filepath = os.path.join(intermediate_directory, 'essay_set1_text_all.txt')

In [12]:
essays = pd.read_csv('../data/intermediate/prepped_essays_df.csv')

In [13]:
essays.head(2)

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6,prompt,has_source_material,source_text,grade_7,grade_8,grade_10
0,1,1,"Dear local newspaper, I think effects computer...",4.0,4.0,NaN,8.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"More and more people use computers, but not ev...",0,NaN,0,1,0
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5.0,4.0,NaN,9.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"More and more people use computers, but not ev...",0,NaN,0,1,0


In [21]:
essays.dropna(axis=1, how='all', inplace=True)

In [25]:
essays = essays[essays['essay_set'] == 1]

In [26]:
len(essays)

1783

In [34]:
intermediate_directory = os.path.join('../data/intermediate')
LDAvis_data_filepath = os.path.join(intermediate_directory, 'ldavis_prepared')

In [27]:
essays.dropna(axis=1, how='all', inplace=True)

In [28]:
essays.head(2)

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,domain1_score,prompt,has_source_material,grade_7,grade_8,grade_10
0,1,1,"Dear local newspaper, I think effects computer...",4.0,4.0,8.0,"More and more people use computers, but not ev...",0,0,1,0
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5.0,4.0,9.0,"More and more people use computers, but not ev...",0,0,1,0


In [29]:
topic_names = {0: 'looking_at_websites_for_info',
               1: 'doesnt_have_the_negative_exercise_effect',
               2: 'spend_time_looking_on_websites',
               3: 'games_and_information',
               4: 'bad_if_kids_spend_too_much_time'}

In [30]:
col_names = pd.DataFrame(columns=list(topic_names.values()))

In [41]:
bigram_model_filepath = os.path.join(intermediate_directory, 'bigram_model_all')
bigram_model = Phrases.load(bigram_model_filepath)
trigram_model_filepath = os.path.join(intermediate_directory, 'trigram_model_all')
trigram_model = Phrases.load(trigram_model_filepath)
trigram_dictionary_filepath = os.path.join(intermediate_directory, 'trigram_dict_all.dict')
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)
lda_model_filepath = os.path.join(intermediate_directory, 'lda_model_all')
lda = LdaMulticore.load(lda_model_filepath)

In [44]:
%timeit

essays_with_topic_scores = process_topic_and_score_df(col_names)

In [45]:
essays_with_topic_scores.head(2)

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,domain1_score,prompt,has_source_material,grade_7,grade_8,grade_10,bad_if_kids_spend_too_much_time,doesnt_have_the_negative_exercise_effect,games_and_information,looking_at_websites_for_info,spend_time_looking_on_websites
0,1,1,"Dear local newspaper, I think effects computer...",4.0,4.0,8.0,"More and more people use computers, but not ev...",0,0,1,0,0,0.54,0.178,0.248,0.031
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5.0,4.0,9.0,"More and more people use computers, but not ev...",0,0,1,0,0.09,0.00,0.000,0.903,0.000


In [46]:
essays_with_topic_scores['length'] = essays_with_topic_scores.essay.str.len()

In [47]:
essays_with_topic_scores.head(2)

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,domain1_score,prompt,has_source_material,grade_7,grade_8,grade_10,bad_if_kids_spend_too_much_time,doesnt_have_the_negative_exercise_effect,games_and_information,looking_at_websites_for_info,spend_time_looking_on_websites,length
0,1,1,"Dear local newspaper, I think effects computer...",4.0,4.0,8.0,"More and more people use computers, but not ev...",0,0,1,0,0,0.54,0.178,0.248,0.031,1875
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5.0,4.0,9.0,"More and more people use computers, but not ev...",0,0,1,0,0.09,0.00,0.000,0.903,0.000,2288


In [48]:
essays_with_topic_scores = essays_with_topic_scores.drop(['essay_set', 'essay', 'rater1_domain1', 'rater2_domain1', 'prompt', 'has_source_material', 'grade_7', 'grade_8', 'grade_10'], axis=1)
# essays_with_topic_scores.to_csv('../data/processed/essays_with_topic_scores.csv')

In [50]:
essays_with_topic_scores.tail(2)

,essay_id,domain1_score,bad_if_kids_spend_too_much_time,doesnt_have_the_negative_exercise_effect,games_and_information,looking_at_websites_for_info,spend_time_looking_on_websites,length
1781,1786,2.0,0.727,0.068,0.068,0.069,0.069,71
1782,1787,7.0,0.979,0.000,0.000,0.000,0.000,1118


In [51]:
essays_with_topic_scores.to_csv('../data/processed/essays_with_topic_scores.csv')